In [1]:
%cd /opt

/opt


In [2]:
%%capture
!tar xvf /kaggle/input/extract-prebuilt-kaldi-from-docker/kaldi.tar

In [3]:
%cd kaldi/egs

/opt/kaldi/egs


In [4]:
!git clone https://github.com/danijel3/ClarinStudioKaldi

Cloning into 'ClarinStudioKaldi'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 778 (delta 0), reused 0 (delta 0), pack-reused 775
Receiving objects: 100% (778/778), 35.26 MiB | 19.10 MiB/s, done.
Resolving deltas: 100% (262/262), done.


In [5]:
%cd ClarinStudioKaldi

/opt/kaldi/egs/ClarinStudioKaldi


In [6]:
%%capture
!conda install -c bioconda perl-perlio-gzip -y

In [7]:
import os
os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib:/opt/kaldi/tools/openfst-1.6.7/lib:/opt/kaldi/src/lib'

In [8]:
!cat path.sh|sed -e 's/~\/apps/\/opt/' > tmp
!mv tmp path.sh

In [9]:
!echo > local_clarin/clarin_pl_clean.sh

In [10]:
!ln -s ../wsj/s5/steps
!ln -s ../wsj/s5/conf
!ln -s ../wsj/s5/local
!ln -s ../wsj/s5/utils

In [11]:
!cp -r /kaggle/input/kaldi-clarinstudio-polish-train-gmm/data /kaggle/working/
!cp -r /kaggle/input/kaldi-clarinstudio-polish-train-gmm/exp /kaggle/working/

In [12]:
!ln -s /kaggle/working/exp
!ln -s /kaggle/working/data

In [13]:
%%writefile run.sh
. path.sh

export nj=40 ##number of concurrent processes
export nj_test=30 ## number of concurrent processes for test has to be <=30

#Performing tests on all the above systems:


#test MMI
steps/decode_fmllr.sh --nj $nj_test exp/tri3b/graph data/test exp/tri3b_mmi/decode

#decode MMI using wider beam
steps/decode_fmllr.sh --nj $nj_test --beam 24 --lattice-beam 12 \
  exp/tri3b/graph data/test exp/tri3b_mmi/decode_wb

#compute the oracle on the last decoding result - to see how much is possible using these lattices
./steps/oracle_wer.sh data/test data/lang exp/tri3b_mmi/decode_wb

#perform rescoring using the large LM in carpa format (much faster than regular arpa)
./steps/lmrescore_const_arpa.sh data/lang_test data/lang_carpa data/test exp/tri3b_mmi/decode_wb exp/tri3b_mmi/decode_rs

find exp -name best_wer | while read f ; do cat $f ; done | sort -k2nr
find exp -name oracle_wer | while read f ; do echo -n "$f: " ; cat $f ; done | sort -k2nr

Overwriting run.sh


In [14]:
# in the last notebook, I tried to zip the symbolic links, forgetting the directory itself was one
!cp exp/tri3b_mmi/4.mdl exp/tri3b_mmi/final.mdl

In [15]:
!bash run.sh

steps/decode_fmllr.sh --nj 30 exp/tri3b/graph data/test exp/tri3b_mmi/decode
steps/decode.sh --scoring-opts  --num-threads 1 --skip-scoring false --acwt 0.083333 --nj 30 --cmd run.pl --beam 10.0 --model exp/tri3b_mmi/final.mdl --max-active 2000 exp/tri3b/graph data/test exp/tri3b_mmi/decode.si
decode.sh: feature type is lda
steps/diagnostic/analyze_lats.sh --cmd run.pl exp/tri3b/graph exp/tri3b_mmi/decode.si
steps/diagnostic/analyze_lats.sh: see stats in exp/tri3b_mmi/decode.si/log/analyze_alignments.log
Overall, lattice depth (10,50,90-percentile)=(1,2,8) and mean=4.1
steps/diagnostic/analyze_lats.sh: see stats in exp/tri3b_mmi/decode.si/log/analyze_lattice_depth_stats.log
local/score.sh --cmd run.pl data/test exp/tri3b/graph exp/tri3b_mmi/decode.si
local/score.sh: scoring with word insertion penalty=0.0,0.5,1.0
steps/decode_fmllr.sh: feature type is lda
steps/decode_fmllr.sh: getting first-pass fMLLR transforms.
steps/decode_fmllr.sh: doing main lattice generation phase
steps/decode_